In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
# os.environ['KERAS_BACKEND']='theano'
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Colab Notebooks/'
path = root+'Signate/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
# reading data
# df = pd.read_csv(path+'train.csv', sep='\t')
df = pd.read_csv(path+'training_sn.tsv', sep='\t')
#df.columns =['sid','sentence','html_id','label']
#df = df.sample(1000).reset_index()
df.head()

,sid,sentence,html_id,label
0,tr-01-0000,PART I,Form10k_01,0
1,tr-01-0001,The “Business” section and other parts of this...,Form10k_01,0
2,tr-01-0002,"Statements that are not historical facts, inc...",Form10k_01,0
3,tr-01-0003,Our actual results may differ materially from...,Form10k_01,0
4,tr-01-0004,Factors that could cause such differences inc...,Form10k_01,0


In [ ]:
#df = df.drop(['sentence_cl','Words','sentence_st'],axis=1)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
set(stopwords.words('english'))
stop_words = set(stopwords.words('english')) 
df["sentence"] = df["sentence"].str.strip()
df["sentence"] = df["sentence"].str.replace('[^\w\s]','')
df["sentence"] = df["sentence"].str.lower()

#df['sentence'].apply(lambda x: [item for item in x word_tokenize(item)])
df['Words']=df['sentence'].apply(lambda x: word_tokenize(x))
df['Words_st']=df['Words'].apply(lambda x: [item for item in x if item not in stop_words])
#df['Words'] = word_tokenize(df['sentence'])
#df["Words"] = df["sentence"].str.lower().str.split()
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
#df['sentence_st'] = df['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#stop = nltk.stopwords.words('english')
#print(stop)
# Get stopwords, stemmer and lemmatizer
#stopwords = nltk.corpus.stopwords.words('english')
#stemmer = nltk.stem.PorterStemmer()
#lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df.head()

,sid,sentence,html_id,label,Words,Words_st
0,tr-01-0000,part i,Form10k_01,0,"[part, i]",[part]
1,tr-01-0001,the business section and other parts of this f...,Form10k_01,0,"[the, business, section, and, other, parts, of...","[business, section, parts, form, 10k, contain,..."
2,tr-01-0002,statements that are not historical facts inclu...,Form10k_01,0,"[statements, that, are, not, historical, facts...","[statements, historical, facts, including, sta..."
3,tr-01-0003,our actual results may differ materially from ...,Form10k_01,0,"[our, actual, results, may, differ, materially...","[actual, results, may, differ, materially, pro..."
4,tr-01-0004,factors that could cause such differences incl...,Form10k_01,0,"[factors, that, could, cause, such, difference...","[factors, could, cause, differences, include, ..."


In [ ]:
print(stop_words)

{"you're", 'wasn', 'not', 'how', 'didn', 'at', 'very', 'mustn', 'are', 'herself', 'nor', 'each', 'y', "shouldn't", 'mightn', 'this', 'they', 'my', 'be', 'because', "should've", 'while', 'shouldn', 'hasn', 's', 'have', 'only', 'yourself', 'all', 'no', 'until', 'aren', 'but', 'doing', 'which', 'haven', 'who', 'isn', 'off', 'further', 'about', "couldn't", 'their', 'some', 'his', 'me', 'there', 'shan', "wouldn't", 'is', 've', 'her', 'now', 'weren', 'the', 'ain', 'did', 'few', 'just', "hadn't", 'couldn', 'she', 'by', 'been', 'yourselves', 'those', 'between', 'above', 'when', 'out', "weren't", 'theirs', "mightn't", 'hers', 'on', 'ma', 'so', 'it', 'to', "that'll", 'them', "you'll", "needn't", 'too', "aren't", "you'd", 'i', 'd', 'before', "you've", 'doesn', 'hadn', "she's", 'you', 'more', 'am', 'than', "isn't", 'wouldn', 'that', 'itself', "didn't", 'yours', 'same', 'once', 'as', 'then', 'has', "hasn't", "shan't", "mustn't", 'o', 't', 'through', 'a', 'after', 'with', 'why', 'can', 'what', 're',

In [ ]:

#df.shape
#df1.shape

In [ ]:
df.shape

(65963, 6)

In [ ]:
nltk.download('wordnet')
stemmer =  nltk.stem.SnowballStemmer('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
#df1['Words_st']=df1['Words_st'].apply(lambda x: lemmatizer.lemmatize(x))
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]
def stem_text(text):
    return [stemmer.stem(w) for w in text]
df['Words'] = df.Words_st.apply(stem_text)    
df['Words_lemmatized'] = df.Words_st.apply(lemmatize_text)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df['Words_lemmatized'])
#df1.drop(['Words_tok'],axis=1)
df['Words_len']=df['Words_lemmatized'].apply(lambda x: len(x))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


AttributeError: ignored

In [ ]:
#df['Words_lemmatized'] = pd.Series(df['Words_lemmatized'])
#df['Words_lemmatized'] = df.Words_st.apply(lemmatize_text)
df1 = df[df['Words_len'] > 3 ]

In [ ]:

df1.head()

,sid,sentence,html_id,label,Words,Words_st,Words_lemmatized,Words_len
1,tr-01-0001,the business section and other parts of this f...,Form10k_01,0,"[busi, section, part, form, 10k, contain, forw...","[business, section, parts, form, 10k, contain,...","[business, section, part, form, 10k, contain, ...",12
2,tr-01-0002,statements that are not historical facts inclu...,Form10k_01,0,"[statement, histor, fact, includ, statement, b...","[statements, historical, facts, including, sta...","[statement, historical, fact, including, state...",17
3,tr-01-0003,our actual results may differ materially from ...,Form10k_01,0,"[actual, result, may, differ, materi, project,...","[actual, results, may, differ, materially, pro...","[actual, result, may, differ, materially, proj...",10
4,tr-01-0004,factors that could cause such differences incl...,Form10k_01,0,"[factor, could, caus, differ, includ, limit, d...","[factors, could, cause, differences, include, ...","[factor, could, cause, difference, include, li...",19
5,tr-01-0005,managements narrative analysis of the results ...,Form10k_01,0,"[manag, narrat, analysi, result, oper, mna, fo...","[managements, narrative, analysis, results, op...","[management, narrative, analysis, result, oper...",8


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
#txt_fitted = tf.fit(df1[['Words_lemmatized']])
#df1['Words_tok'] = txt_fitted.transform(df1[['Words_lemmatized']])
#df1['Words_tok']=df1['Words_lemmatized'].apply(lambda x: txt_fitted.transform(x))
df1['Words_lemmatized'] = ''.join(str(s) for s in df1['Words_lemmatized'])
#df1['Words_tok']=df1['Words_lemmatized'].apply(lambda x: text_to_word_sequence(x))
df1['Words_tok']=df1['Words_lemmatized'].apply(lambda x: tokenizer.texts_to_sequences(x))


#df1['Words_tok']=df1['Words_lemmatized'].apply(lambda x: tokenizer.(x))
#df1['Words_tok'] = pd.Series(df1['Words_tok'])
df1.head()

#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer()
#vectorizer.fit(df1[['Words_lemmatized']])
#df1['Words_cv'] = vectorizer.transform(df1['Words_lemmatized'])


In [ ]:
df1['Words_tok'] 
df1[df1['Words_len']>50]
#import seaborn as sns
#import matplotlib.pyplot as plt
#sns.countplot(x ='Words_len', data = df1)
  
# Show the plot
#plt.show()
df1['Words_tok']

1         
2         
3         
4         
5         
        ..
65946     
65949     
65952     
65955     
65958     
Name: Words_tok, Length: 58316, dtype: object

In [ ]:
df1.shape

In [ ]:
df_model = df1[['sid','Words_tok','label']]

In [ ]:
word_index = tokenizer.word_index
unique_tokens = len(word_index)
#understandig purpose only
print(word_index)
print('Number of Unique Tokens',unique_tokens)

In [ ]:
df_model

In [ ]:
#df_model['Words_pad']=df_model['Words_tok'].apply(lambda x: pad_sequences(x , maxlen=10))

In [ ]:
df_model['Words_pad'] = tf.keras.preprocessing.sequence.pad_sequences(df_model['Words_tok'])